# Motivation

The scikit package for python will be one of the most important ones during the master program. Therefore I came up with the idea to get a overview what topics are coverd on the documentation webpage and especially which external resources are provided. The apporoch would be to start at the highest level of the webpage and then go down the tree shaped website directory. I limit the analysis to 2 levels below the starting point, otherwise it would be computational challenging. 

### Packages

In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import queue
from concurrent.futures import ThreadPoolExecutor
from urllib.parse import urlparse

### Define classes and functions needed

In [2]:
def get_class(DRs):
    try:
        if len(DRs.get("class")) > 1: 
            class_lap = str("")
            for i in range(len(DRs.get("class"))):
                class_lap = str(class_lap + DRs.get("class")[i])
            return class_lap
        else: 
            return DRs.get("class")
    except:
        "Not Labeled"

In [3]:
def website_type(webdir):
    try: 
        if (webdir[0:4] == "http") or (webdir[-3:] == "pdf") or (webdir[0:2] == ".."):
            return "foreign"
        else: 
            return "scikit"
    except: 
        return "foreign"

In [4]:
def get_links_scikit(webdir):
    if website_type(webdir) == "scikit":
        URL = "http://scikit-learn.org/stable/"
        # webdir = "documentation.html"
        next_url = str(URL + webdir)
        soup = BeautifulSoup(requests.get(next_url).content, "lxml")
        link_data = pd.DataFrame([])
        if len(soup.select("div.body a")) > 0:
            for DRs in soup.select("div.body a"):
                link_data = link_data.append(pd.DataFrame({"Website": URL, "Location": webdir, 
                                                           "Links": DRs.get("href"), "Type": get_class(DRs)}, 
                                                          index=[0])
                                             , ignore_index=True)
        else: 
            link_data = pd.DataFrame({"Website": URL, "Location": webdir, "Links": "No Link here", "Type": "Not Labeled"}, 
                                                      index=[0])
    else: 
        link_data = pd.DataFrame({"Website": webdir, "Location": webdir, "Links": "No Link here", "Type": "Not Labeled"}, 
                                                      index=[0])
    return link_data

In [5]:
def simple_worker(i):
    while True:
        item = q.get()
        if item == 'break':
            break
        results = get_links_scikit(item)
        q.task_done()
        if (q.unfinished_tasks in [2000,1500,1000,500,10]) or (q.unfinished_tasks < 10):
            print("Task_done & amount of unfinished sub tasks: " + str(q.unfinished_tasks))
        return results

In [6]:
def hard_worker(i):
    while True:
        item = m.get()
        if item == 'break':
            break
        results = get_links_scikit(item)
        if any(results):
            for x in results["Links"]:
                q.put(x)
        m.task_done()
        print("Task_done & amount of unfinished tasks: " + str(q.unfinished_tasks))
        return results

### Retrieving information from the website

In [7]:
m = queue.Queue()
q = queue.Queue()
initial_tasks = list(get_links_scikit("documentation.html")["Links"])
for y in initial_tasks:
    m.put(y)
with ThreadPoolExecutor(max_workers=4) as executor:
    m_results = executor.map(hard_worker,range(m.unfinished_tasks))
    m.join()
    for i in range(4):
        m.put('break')

Task_done & amount of unfinished tasks: 1
Task_done & amount of unfinished tasks: 2
Task_done & amount of unfinished tasks: 49
Task_done & amount of unfinished tasks: 80
Task_done & amount of unfinished tasks: 121
Task_done & amount of unfinished tasks: 209
Task_done & amount of unfinished tasks: 773
Task_done & amount of unfinished tasks: 801
Task_done & amount of unfinished tasks: 851
Task_done & amount of unfinished tasks: 852
Task_done & amount of unfinished tasks: 1331
Task_done & amount of unfinished tasks: 2004
Task_done & amount of unfinished tasks: 2080


In [8]:
with ThreadPoolExecutor(max_workers=25) as executor:
    futures = executor.map(simple_worker,range(q.unfinished_tasks))
    q.join()
    for i in range(25):
        q.put('break')

Task_done & amount of unfinished sub tasks: 2000
Task_done & amount of unfinished sub tasks: 1500
Task_done & amount of unfinished sub tasks: 1000
Task_done & amount of unfinished sub tasks: 500
Task_done & amount of unfinished sub tasks: 10
Task_done & amount of unfinished sub tasks: 9
Task_done & amount of unfinished sub tasks: 8
Task_done & amount of unfinished sub tasks: 7
Task_done & amount of unfinished sub tasks: 6
Task_done & amount of unfinished sub tasks: 5
Task_done & amount of unfinished sub tasks: 4
Task_done & amount of unfinished sub tasks: 3
Task_done & amount of unfinished sub tasks: 2
Task_done & amount of unfinished sub tasks: 1
Task_done & amount of unfinished sub tasks: 0


In [9]:
links_full = pd.DataFrame([])
for value in m_results:
    links_full = links_full.append(value,ignore_index=True)
for value in futures:
    links_full = links_full.append(value,ignore_index=True)

### Data Manipulation

In [10]:
links_full["url"] = links_full.Links.apply(lambda x: urlparse(x).netloc)

In [11]:
# Find all youtube links from scikit
links_full[links_full['url'].str.contains('youtube', na = False)].Links

1997     https://www.youtube.com/watch?v=Zd5dfooZWG4
2001     https://www.youtube.com/watch?v=cHZONQ2-x7I
11651    https://www.youtube.com/watch?v=Jm-eBD9xR3w
Name: Links, dtype: object

### Saving data

In [12]:
links_full.to_csv("scikit_FullLinks.csv")